# Step 1: Understand the business problem.

1. URL - Website of the Zomato for each restaurant. - Object datatype
2. Address - Address of the Restaurant. - Object datatype
3. Name - Name of the restaurant. - Object datatype
4. Online Order - The customer ordered the menu online or not. - Object datatype
5. Book table - The customer has booked the table or not. - Object datatype
6. Rate - Rating of the restaurant that has by the customer. - Numerical datatype
7. Votes - The votes have been given by the customer to the restaurant. - Numerical 
datatype
8. Phone - Contact number of the Restaurant. - Object datatype
9. Location - The city name where the restaurant is located. - Object datatype
10. Rest Type - The type of restaurant. - Object datatype
11. Dish liked - Dishes liked by the customer from the restaurant. - Object datatype
12. Cuisines - The cuisines that have been prepared by the restaurant. - Object datatype
13. Approx Cost for two people - The approximate cost of the customer for 2 people. -
Number datatype
14. Reviews list - The reviews made by the customers on the restaurant. - Object 
datatype
15. Menu Item - The menu items that are usually available at the restaurant. - Object 
datatype
16. Listed in (type) - Contains the type of the meal. - Object datatype
17. Listed in (city) - This contains the neighborhood in which the restaurant is listed. -
Object datatype

In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# display all columns of the dataframe
pd.options.display.max_columns = None

# display all rows of the dataframe
pd.options.display.max_rows = None
 
# to display the float values upto 6 decimal places     
pd.options.display.float_format = '{:.6f}'.format
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.linear_model import LinearRegression

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler 

# import SGDRegressor from sklearn to perform linear regression with stochastic gradient descent
from sklearn.linear_model import SGDRegressor

# import function for ridge regression
from sklearn.linear_model import Ridge

# import function for lasso regression
from sklearn.linear_model import Lasso

# import function for elastic net regression
from sklearn.linear_model import ElasticNet

# import function to perform GridSearchCV
from sklearn.model_selection import GridSearchCV

from mlxtend.feature_selection import SequentialFeatureSelector as sfs

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
import re

# step 2: Read the data, and convert the data types.
Note: The data set has numerical and categorical data but due to noise(anomaly) in 
the data, the columns are treated as the object type. And You may feel like converting the 
features into numerical at this step if not appropriate at this stage, In that case, feel free to 
convert the variable to the appropriate type in the further step as well based on your way 
of analyzing the data.


In [ ]:
zomato_orgnl=pd.read_csv('zomato.csv')
zomato_orgnl.head()

In [ ]:
zomato_orgnl.shape

In [ ]:
zomato_orgnl.info()

In [ ]:
zomato=zomato_orgnl.drop(['url','dish_liked','phone','phone'],axis=1)

In [ ]:
#Changing the Columns Names
zomato_orgnl.columns
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type','listed_in(city)':'city'})

# Cleaning the dataset

In [ ]:
zomato['cost'].unique()

In [ ]:
zomato['cost'] = zomato['cost'].astype(str)
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.'))
zomato['cost'] = zomato['cost'].astype(float)
zomato.info()

In [ ]:
#Reading uninque values from the Rate column
zomato['rate'].unique()

In [ ]:
zomato['rate'].unique()
zomato = zomato.loc[zomato.rate !='NEW']
zomato = zomato.loc[zomato.rate !='-'].reset_index(drop=True)

remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')
zomato['rate'].head()

Step 3: Perform the described method for the data, Try to find any essential points from the 
described analysis. And check the missing values and Duplicate records. Impute the 
missing values in the best way possible.
Note: To impute the missing values with parameters, You must find the best parameter.
Hints:
● Check the distribution using plots. And check the Skewness, Kurtosis and etc.

In [ ]:
zomato.describe()

In [ ]:
zomato.describe(include=object)


In [ ]:
zomato.isnull().sum()

In [ ]:
zomato.isnull().sum()/zomato.isnull().count()*100

In [ ]:
zomato['rate'].fillna(zomato['rate'].mode()[0],inplace=True)
zomato['rest_type'].fillna(zomato['rest_type'].mode()[0],inplace=True)
zomato['cost'].fillna(zomato['cost'].mean(),inplace=True)

# Drop null values

In [ ]:
#Remove the NaN values from the dataset
zomato.dropna(how='any',inplace=True)
zomato.isnull().sum()

In [ ]:
df_num=zomato.select_dtypes(include=np.number)
df_num.columns

In [ ]:
t=1
plt.figure(figsize=(15,10))
for i in df_num:
    plt.subplot(2,2,t)
    sns.distplot(zomato[i])
    plt.xticks(rotation=90)
    t+=1
plt.show()
plt.tight_layout()

In [ ]:
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

In [ ]:
zomato.shape

# Step 4: Once the basic preprocessing is done like converting the data types, missing value  imputation, and duplicate rows. perform the EDA(Exploratory Data Analysis) on the data to  find the various factors that will help to understand the cost per two persons.

In [ ]:
df_cat=zomato.select_dtypes(include=object)
df_cat.columns

In [ ]:
sns.countplot(zomato['online_order'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Restaurants delivering online or Not')

# How many restaurants are present at different city?

In [ ]:
zomato.city.value_counts()

In [ ]:
# Countplot of city

plt.figure(figsize=(20,10))
sns.countplot(x='city', data=zomato)
plt.title('Count of Restaurants at each Location', fontsize=20, fontweight='bold')
plt.xticks(rotation=90)
plt.show()

There are more than 3000 restaurants in BTM. Quick Google Search shows us that BTM is posh residential area so because of that there are quite a lot of restaurants and also it is famous for cafes. JP Nagar HSR, Koramangala 5th block, Whitefield, Indiranagar have more than 2000 restaurants and Jayanagar, Marathahalli, Bannerghatta Road have more than 1000 restaurants.

# Restaurant types

In [ ]:
# Bar plot of rest_type

plt.figure(figsize=(20,10))
sns.countplot(x='rest_type', data=zomato)
plt.xticks(rotation=90)
plt.title('Count of each restaurant type', fontsize=20, fontweight='bold')
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
chains = zomato['name'].value_counts()[:20]
sns.barplot(x=chains, y=chains.index, palette='Set1')
plt.xlabel("Number of outlets", size=15)
plt.ylabel("Name of Restaurants", size=15)
plt.title("Most famous restaurant chains", fontsize=20, fontweight='bold')
plt.show()

In [ ]:
# Cafe Coffee Day has maximum outlets in the city and is followed by Onesta with number of outlets little more than 80. It can be seen that most famous restaurant chains have more than 50 outlets.

In [ ]:
df_online_booking = zomato.groupby(["online_order", "book_table"]).size().reset_index(name = "Count")
plt.figure(figsize = (15,7))
sns.barplot(x="online_order", y="Count", hue="book_table", data=df_online_booking)
plt.title("Online Order - Book Table", fontsize=20, fontweight='bold')
plt.show()

In [ ]:
label1 = ["Accepted",'Not Accepted']
label2 = ['Not Accepted', "Accepted"]

df_online = zomato['online_order'].value_counts().values
df_table = zomato['book_table'].value_counts().values

fig, ax1 = plt.subplots(nrows=1, ncols=2)

color1 = ['#FA744F', '#16817A']
color2 = ['#FC5050', '#FFD00C']

ax1[0].pie(df_online, labels=label1, autopct='%1.1f%%', startangle=90, colors=color1)
ax1[0].set_title('Online order', fontsize=20, fontweight='bold')

ax1[1].pie(df_table, labels=label2, autopct='%1.1f%%', startangle=90, colors=color2)
ax1[1].set_title('Book table', fontsize=20, fontweight='bold')

In [ ]:
plt.rcParams['figure.figsize'] = (13, 9)
Y = pd.crosstab(zomato['rate'], zomato['book_table'])
Y.div(Y.sum(1).astype(float), axis = 0).plot(kind = 'bar', stacked = True,color=['red','yellow'])
plt.title('table booking vs rate', fontweight = 30, fontsize = 20)
plt.legend(loc="upper right")
plt.show()

In [ ]:
sns.countplot(zomato['city'])
sns.countplot(zomato['city']).set_xticklabels(sns.countplot(zomato['city']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(13,13)
plt.title('Location')

In [ ]:
loc_plt=pd.crosstab(zomato['rate'],zomato['city'])
loc_plt.plot(kind='bar',stacked=True);
plt.title('Location - Rating',fontsize=15,fontweight='bold')
plt.ylabel('Location',fontsize=10,fontweight='bold')
plt.xlabel('Rating',fontsize=10,fontweight='bold')
plt.xticks(fontsize=10,fontweight='bold')
plt.yticks(fontsize=10,fontweight='bold');
plt.legend().remove();

In [ ]:
plt.figure(figsize=(12,6)) 
zomato_orgnl['location'].value_counts()[:10].plot(kind = 'pie')
plt.title('Location', weight = 'bold')

In [ ]:
plt.figure(figsize = (12,6))
names = zomato_orgnl['location'].value_counts()[:10].index
values = zomato_orgnl['location'].value_counts()[:10].values
colors = ['gold', 'red', 'lightcoral', 'lightskyblue','blue','green','silver']
explode = (0.1, 0, 0, 0,0,0,0,0,0,0)  # explode 1st slice

plt.pie(values, explode=explode, labels=names, colors=colors,autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title("Percentage of restaurants present in that location", weight = 'bold')
plt.show()

# Distribution of Cost of Food for two People

In [ ]:
#zomato.menu_item = np.where(zomato.menu_item == '[]',zomato.menu_item.value_counts().index[1],zomato.menu_item)

# Step 5: Make a copy of the data set and Perform the preprocessing that require for the  model.Note: You can see many categorical variables with a high number of unique values. Therefore do not keep dropping the variables as the first option, try to create new variables  or perform any other feature engineering methods.

In [ ]:
df_zomato=zomato.copy()

#  Convert the online categorical variables into a numeric format

In [ ]:
zomato.online_order[zomato.online_order == 'Yes'] = 1 
zomato.online_order[zomato.online_order == 'No'] = 0

In [ ]:
zomato.online_order.value_counts()

In [ ]:
zomato.online_order = pd.to_numeric(zomato.online_order)

# change the string categorical into to a categorical int

In [ ]:
zomato.book_table[zomato.book_table == 'Yes'] = 1 
zomato.book_table[zomato.book_table == 'No'] = 0

In [ ]:
zomato.book_table = pd.to_numeric(zomato.book_table)

In [ ]:
zomato.book_table.value_counts()

# Label encode the categorical variables to make it easier to build algorithm

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
zomato.location = le.fit_transform(zomato.location)
zomato.rest_type = le.fit_transform(zomato.rest_type)
zomato.cuisines = le.fit_transform(zomato.cuisines)
zomato.menu_item = le.fit_transform(zomato.menu_item)

In [ ]:
# #def Encode(zomato):
#     for column in zomato.columns[~zomato.columns.isin(['rate', 'cost', 'votes'])]:
#         zomato[column] = zomato[column].factorize()[0]
#     return zomato

# zomato_en = Encode(zomato.copy())

In [ ]:
#zomato['Number_of_cuisines_offered'] = zomato['cuisines'].apply(lambda x : len(x.split(',')))

In [ ]:
#df_numerical=zomato.select_dtypes(include=np.number)
#df_categorical=zomato.select_dtypes(include=object)

In [ ]:
#df_numerical.columns

In [ ]:
#df_categorical.columns

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#X_scaler = MinMaxScaler()

#num_scaled = X_scaler.fit_transform(df_numerical)

#df_num_scaled = pd.DataFrame(num_scaled, columns = df_numerical.columns)

In [ ]:
#len(df_categorical.city.unique())

In [ ]:
#df_dummy=pd.get_dummies(data=df_categorical,columns=['book_table','online_order','type'],drop_first=True)

In [ ]:
#df_dummy['cuisines']=df_dummy['cuisines'].replace(df_dummy.cuisines.value_counts(1))
#df_dummy['name']=df_dummy['name'].replace(df_dummy.name.value_counts(1))
#df_dummy['rest_type']=df_dummy['rest_type'].replace(df_dummy.rest_type.value_counts(1))
#df_dummy['city']=df_dummy['city'].replace(df_dummy.city.value_counts(1))

In [ ]:
#zomato = pd.concat([df_num_scaled,df_dummy], axis = 1)

In [ ]:
zomato.head()

# Step 6: Perform statistical hypothesis testing on features to get an idea of whether features are impacting the target variables.

Dividing the dataset into categorical and numerical columns and then finding pvalue for each independent feature corresponding to dependent feture(cost). Keeping pvalue threshold as 0.05

1. Hypothesis Testing for Categorical columns and stroke

Hypothesis :

H0:Independent variables are not significantly associated with the dependent variable (cost)

H1:Independent variables are significantly associated with the dependent variable (cost)

if pvalue<0.05 we will reject the null hypothesis.

In [ ]:
cat_cols = ['name', 'online_order', 'book_table', 'rest_type', 'cuisines','reviews_list', 'type', 'city']

In [24]:
# To perform hypothesis between categorical columns we are using chi2_contingency

In [25]:
from scipy.stats import chi2_contingency
pvalue_score = pd.DataFrame(columns=['Feature','pvalue'])


def update_score_categorical(cat_cols):
    
    global pvalue_score
    for col in cat_cols:
        contingency_table = pd.crosstab(zomato[col], zomato['cost'])
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        pvalue_score = pvalue_score.append({'Feature':col,'pvalue':p_value}, ignore_index = True)

In [26]:
# list of categorical variables
cat_cols = ['name', 'online_order', 'book_table', 'rest_type', 'cuisines','reviews_list', 'type', 'city']
update_score_categorical(cat_cols)


2. Hypothesis Testing for numerical columns and stroke

Hypothesis :

H0:The variables are not correlated with each other

H1:The variables are correlated with each other

if pvalue<0.05 we will reject the null hypothesis.

In [27]:
pvalue_score

,Feature,pvalue
0,name,0.000000
1,online_order,0.000000
2,book_table,0.000000
3,rest_type,0.000000
4,cuisines,0.000000
5,reviews_list,0.000000
6,type,0.000000
7,city,0.000000


# Step 7: Split the dataset into train and test data sets and Perform the scaling on both sets if necessary

In [28]:
from sklearn.model_selection import train_test_split 
x = zomato.iloc[:,[2,3,4,5,6,7,8,11]]
y = zomato['cost']
#Getting Test and Training Set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1,random_state=353)
print(x_train.shape)
print(y_train.shape)

(44496, 8)
(44496,)


In [29]:
# from sklearn.model_selection import train_test_split 
# x = zomato_en.iloc[:,[2,3,4,5,6,7,8,11,12]]
# y = zomato_en['cost']
# #Getting Test and Training Set
# x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1,random_state=353)
# x_train.head()
# y_train.head()

# Step 8: Build the base model.

# LinearRegression

In [30]:
lin_reg=LinearRegression()
model=lin_reg.fit(x_train,y_train)
ypred=model.predict(x_test)
rmse=np.sqrt(mean_squared_error(y_test,ypred))
print(rmse)
from sklearn.metrics import r2_score
r2=r2_score(y_test,ypred)
print(r2)


ValueError: could not convert string to float: 'Yes'

# Descision tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1,random_state=105)
Dtree=DecisionTreeRegressor(min_samples_leaf=.0001)
Dtree.fit(x_train,y_train)
y_predict=Dtree.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

# RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
Rforest=RandomForestRegressor(n_estimators=500,random_state=10,min_samples_leaf=.0001)
Rforest.fit(x_train,y_train)
y_predict=Rforest.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

In [ ]:
Randpred =pd.DataFrame({ "actual": y_test, "pred": y_predict })
Randpred.head()

# Step 9: Understand how the model is performing, Perform feature engineering again if needed. Do feature selection. Try with various models like a parametric and nonparametric models. Once you choose the final model, rebuild the model with best parameters. Note: If you are performing with Linear models, check the model is fulfilling the assumptions

In [ ]:
def get_train_rmse(model): 
    train_pred = model.predict(x_train)
    mse_train = mean_squared_error(y_train, train_pred)
    rmse_train = round(np.sqrt(mse_train), 4)
    return(rmse_train)

In [ ]:
def get_test_rmse(model):
    test_pred = model.predict(x_test)
    mse_test = mean_squared_error(y_test, test_pred)
    rmse_test = round(np.sqrt(mse_test), 4)
    return(rmse_test)

In [ ]:
def mape(actual, predicted):
    return (np.mean(np.abs((actual - predicted) / actual)) * 100)
def get_test_mape(model):
    test_pred = model.predict(x_test)
    mape_test = mape(y_test, test_pred)
    return(mape_test)

In [ ]:
def get_score(model):
    r_sq = model.score(x_train, y_train)
    n = x_train.shape[0]
    k = x_train.shape[1]
    r_sq_adj = 1 - ((1-r_sq)*(n-1)/(n-k-1))
    return ([r_sq, r_sq_adj])

In [ ]:
score_card = pd.DataFrame(columns=['Model_Name', 'Alpha (Wherever Required)', 'l1-ratio', 'R-Squared',
                                       'Adj. R-Squared', 'Test_RMSE', 'Test_MAPE'])
def update_score_card(algorithm_name, model, alpha = '-', l1_ratio = '-'):
    global score_card
    score_card = score_card.append({'Model_Name': algorithm_name,
                       'Alpha (Wherever Required)': alpha, 
                       'l1-ratio': l1_ratio, 
                       'Test_MAPE': get_test_mape(model), 
                       'Test_RMSE': get_test_rmse(model), 
                       'R-Squared': get_score(model)[0], 
                       'Adj. R-Squared': get_score(model)[1]}, ignore_index = True)

In [ ]:
sgd = SGDRegressor(random_state = 10)
linreg_with_SGD = sgd.fit(x_train, y_train)
print('RMSE on train set:', get_train_rmse(linreg_with_SGD))
print('RMSE on test set:', get_test_rmse(linreg_with_SGD)) 

In [ ]:
update_score_card(algorithm_name = 'Linear Regression (using SGD)', model = linreg_with_SGD)
score_card

In [ ]:
ridge = Ridge(alpha = 1, max_iter = 500)
ridge.fit(x_train, y_train)
print('RMSE on test set:', get_test_rmse(ridge))

In [ ]:
update_score_card(algorithm_name='Ridge Regression (with alpha = 1)', model = ridge, alpha = 1)
score_card

In [ ]:
ridge = Ridge(alpha = 2, max_iter = 500)
ridge.fit(x_train, y_train)
print('RMSE on test set:', get_test_rmse(ridge))

In [ ]:
update_score_card(algorithm_name = 'Ridge Regression (with alpha = 2)', model = ridge, alpha = '2')
score_card

In [ ]:
lasso = Lasso(alpha = 0.01, max_iter = 500)
lasso.fit(x_train, y_train)
print('RMSE on test set:', get_test_rmse(lasso))

In [ ]:
update_score_card(algorithm_name = 'Lasso Regression', model = lasso, alpha = '0.01')
score_card

In [ ]:
enet = ElasticNet(alpha = 0.1, l1_ratio = 0.01, max_iter = 500)
enet.fit(x_train, y_train)
print('RMSE on test set:', get_test_rmse(enet))

In [ ]:
update_score_card(algorithm_name = 'Elastic Net Regression', model = enet, alpha = '0.1', l1_ratio = '0.01')
score_card

In [ ]:
tuned_paramaters = [{'alpha':[1e-15, 1e-10, 1e-8, 1e-4,1e-3, 1e-2, 0.1, 1, 5, 10, 20, 40, 60, 80, 100]}]
ridge = Ridge()
ridge_grid = GridSearchCV(estimator = ridge, 
                          param_grid = tuned_paramaters, 
                          cv = 10)
ridge_grid.fit(x_train, y_train)
print('Best parameters for Ridge Regression: ', ridge_grid.best_params_, '\n')
print('RMSE on test set:', get_test_rmse(ridge_grid))

In [ ]:
update_score_card(algorithm_name = 'Ridge Regression (using GridSearchCV)', 
                  model = ridge_grid, 
                  alpha = ridge_grid.best_params_.get('alpha'))
score_card

In [ ]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
tuned_paramaters = [{'alpha':[1e-15, 1e-10, 1e-8, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20]}]
lasso = Lasso()
lasso_grid = GridSearchCV(estimator = lasso, 
                          param_grid = tuned_paramaters, 
                          cv = cv)
lasso_grid.fit(x_train, y_train)
print('Best parameters for Lasso Regression: ', lasso_grid.best_params_, '\n')
print('RMSE on test set:', get_test_rmse(lasso_grid))

In [ ]:
update_score_card(algorithm_name = 'Lasso Regression (using GridSearchCV)', 
                  model = lasso_grid, 
                  alpha = lasso_grid.best_params_.get('alpha'))
score_card

In [ ]:
tuned_paramaters = [{'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20, 40, 60],
                      'l1_ratio':[0.0001, 0.0002, 0.001, 0.01, 0.1, 0.2]}] 
enet = ElasticNet()
enet_grid = GridSearchCV(estimator = enet, 
                          param_grid = tuned_paramaters, 
                          cv = 10)
enet_grid.fit(x_train, y_train)
print('Best parameters for Elastic Net Regression: ', enet_grid.best_params_, '\n')
print('RMSE on test set:', get_test_rmse(enet_grid))

In [ ]:
update_score_card(algorithm_name = 'Elastic Net Regression (using GridSearchCV)', 
                  model = enet_grid, 
                  alpha = enet_grid.best_params_.get('alpha'), 
                  l1_ratio = enet_grid.best_params_.get('l1_ratio'))
score_card

In [ ]:
score_card = score_card.sort_values('Test_RMSE').reset_index(drop = True)
score_card.style.highlight_min(color = 'lightblue', subset = 'Test_RMSE')

We can see that Lasso Regression (using GridSearchCV) has the lowest test RMSE ans test MAPE. 

In [ ]:
lasso = Lasso(alpha = 0.0000000000000001, max_iter = 500)
# lasso.fit(x_train, y_train)


In [ ]:
lasso_forward = sfs(estimator = lasso, k_features = 'best', forward = True,
                     verbose = 2, scoring = 'r2')

In [ ]:
abc = lasso_forward.fit(x_train, y_train)
# print('RMSE on test set:', get_test_rmse(lasso_forward))

In [ ]:
# print the selected feature names when k_features = 12
print('Features selelected using forward selection are: ')
print(abc.k_feature_names_)

# print the R-squared value
print('\nR-Squared: ', abc.k_score_)

# Step 10: Based on your understanding of the model and EDA analysis, Explain the business understanding.


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

In [ ]:
scaler = StandardScaler()
lasso = Lasso()
selector = SelectFromModel(lasso)
selector.fit_transform(x_train, y_train)

In [ ]:
    param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}
    grid = GridSearchCV(Lasso(), param_grid=param_grid, cv=5)
    grid.fit(x_train, y_train)
    alpha = grid.best_params_['alpha']

    # Train the Lasso regression model with the optimal hyperparameters
    lasso = Lasso(alpha=alpha)
    lasso.fit(x_train, y_train)

    # Calculate the R-squared value for the model
    r_squared = lasso.score(x_train, y_train)